# Collecting game-level data from the Lithuanian Basketball League website

In this notebook, I collect all information about all games that have ever taken in the Lithuanian Basketball League since season 1993-1994, including the games played in the King Mindaugas Cup. All the information is saved into a SQLite database for further processing. 

In [ ]:
%load_ext watermark

In [207]:
import requests 
from bs4 import BeautifulSoup
import lxml
import re
import itertools as itt
import pandas as pd
from collections import defaultdict
from tqdm import tqdm
import sqlite3 as sq

DB_PATH = '../data/data.sqlite.db'

%watermark --iversions

lxml    : 4.8.0
pandas  : 1.4.2
requests: 2.27.1
sqlite3 : 2.6.0
sys     : 3.9.12 | packaged by conda-forge | (main, Mar 24 2022, 23:22:55) 
[GCC 10.3.0]
re      : 2.2.1



## Data collection

First, I get a list of teams participating in the Lithuanian Basketball League in the season 2021-2022, with their IDs and seasons the teams participated in.

In [174]:
r = requests.get("https://lkl.lt/turnyrine-lentele")
teams = {tag.text.strip(): {'homepage':  tag["href"], 'name': tag.text.strip()} for tag in BeautifulSoup(r.text, 'lxml').find_all('a', class_='team-title')}

In [175]:
def update_info(html, team):
    
    #get ID
    pattern = re.compile("window.teamId = (\d+);")
    match = pattern.search(html)
    if match:
        team['id'] = match.group(1)
    else:
        print("Warning - no team ID found for %s" % team['name'])
    
    #get seasons
    team['seasons'] = {}
    seasons = [s.find_all("option") for s in BeautifulSoup(html, 'lxml').find_all(class_="season-id")]
    if not seasons:
        print("Warning - no seasons found for %s" % team['name'])
    for season in itt.chain(*seasons):
        team['seasons'][season.text.strip()] = season['value']
    

for name, team in tqdm(teams.items()):
    homepage = requests.get(team['homepage'])
    update_info(homepage.text, team)
    

Then, I collect all seasons observed with their associated IDs.

In [84]:
seasons = defaultdict(lambda: defaultdict(int))

for season, id in itt.chain(*[team['seasons'].items() for team in teams.values()]):
    seasons[season]['participants'] += 1
    if seasons[season]['id'] == 0:
        seasons[season]['id'] = id
    elif seasons[season]['id'] != id:
        print("Season %s has multiple IDs in data! %s and %s" % season, id, seasons[season]['id'])
    seasons[season]['name'] = season


Finally, I go through all the seasons and collect all individual game data, recording the location, score, home and away teams

In [126]:
def parse_result_page(html, url, season_id):
    gameid_pattern = re.compile("https\:\/\/lkl\.lt\/rungtynes\/(\d+)")
    gamescore_pattern = re.compile("(\d+) - (\d+)")

    games = []
    for game in html.find_all("div", class_="result-item"):
        gameinfo = {"season_id" : season_id}
        
        #get location
        gameinfo['location'] = game.find("div", class_="location").text.strip()
        
        #get game id 
        result = game.find("span", class_="result").find("a")
        idmatch = gameid_pattern.search(result['href'])
        if idmatch:
            gameinfo['game-id'] = idmatch.group(1)
        else: 
            print("No game ID found for game %s (page %s)" % result['href'], url)
        
        #get game score         
        score_match = gamescore_pattern.search(result.text)
        if score_match:
            gameinfo['home-points'] = int(score_match.group(1))
            gameinfo['away-points'] = int(score_match.group(2))
        else:
            print("No game score found for game %s (page %s)" % result.text, url)

        #get teams
        links = game.find("div", class_="battle-row").find_all("a")
        gameinfo['home-team-url'] = links[0]['href']
        gameinfo['home-team-name'] = links[0].text.strip()

        gameinfo['away-team-url'] = links[4]['href']
        gameinfo['away-team-name'] = links[4].text.strip()
        games.append(gameinfo)
    
    return games

def failsafe_int(text):
    try: 
        return int(text)
    except ValueError: 
        return -1


all_games = []
for season in tqdm(seasons.values()):
    #get the first page of the results 
    params = {
        "team": "-",
        "month": "-",
        "season" : season['id'],
        "page" : 1
    }
    firstpage = requests.get("https://lkl.lt/loadResults", params=params)
    html = BeautifulSoup(firstpage.text, 'lxml')
    
    #parse the games in the first page
    games = parse_result_page(html=html, url = firstpage.url, season_id=season['id'])
    all_games += games
    
    #get number of pages
    page_numbers = [failsafe_int(p.text) for p in html.find_all(class_= "page-link")]
    if page_numbers:
        max_page = max(page_numbers)

        #parse all subsequent pages 
        for pageNo in range(2, max_page + 1):
            params['page'] = pageNo
            page = requests.get("https://lkl.lt/loadResults", params=params)
            html = BeautifulSoup(page.text, 'lxml')
            games = parse_result_page(html=html, url = page.url, season_id=season['id'])
            all_games += games    

100%|██████████| 36/36 [03:04<00:00,  5.11s/it]


## Organizing collected information

In this section I:
 - Add team abbreviations to the known team list
 - Add teams that are observed in the historical data but no longer play in the championship to the team list
 - Reorganize data into a better structure and save to a SQLite database

### Adding team abbreviations to current teams

In [176]:
home_teams = set([(g['home-team-url'], g['home-team-name']) for g in all_games])
away_teams = set([(g['away-team-url'], g['away-team-name']) for g in all_games])
all_teams = home_teams.union(away_teams)

team_dict = dict(all_teams)
for team in teams.values():
    team['abbreviation'] = team_dict[team['homepage']]
    team['current_team'] = True


### Adding missing teams to the list (and giving them "fake" IDs for consistency)

In [178]:
all_current_teams = set([(t['homepage'], t['abbreviation']) for t in teams.values()])
no_longer_playing_teams = all_teams.difference(all_current_teams)

i = 999
for team_url, team_abbr in no_longer_playing_teams:
    name = team_abbr + "_" + str(i)
    teams[name] = {
        'current_team': False,
        'homepage': team_url,
        'abbreviation': team_abbr,
        'id' : i,
        'name': name,
    }
    i += 1

### Organizing all the data and saving it

In [204]:
teams_df = pd.DataFrame(teams.values())
games_df = pd.DataFrame(all_games)
seasons_df = pd.DataFrame(seasons.values())

teams_df.drop(['seasons'], axis=1, inplace=True)
seasons_df.drop(['participants'], axis=1, inplace=True)

#### Organizing games

In [205]:
with_home = pd.merge(games_df, teams_df, left_on=['home-team-url', 'home-team-name'], right_on=['homepage', 'abbreviation']).rename({"id": "home-team-id"}, axis=1)
with_away = pd.merge(with_home, teams_df, left_on=['away-team-url', 'away-team-name'], right_on=['homepage', 'abbreviation']).rename({"id": "away-team-id"}, axis=1)

cols_of_interest = ['season_id', 'location', 'game-id', 'home-points', 'away-points', 'home-team-id', 'away-team-id']
clean_game_df = with_away[cols_of_interest]

#### Parsing season information

In [224]:
yrs = seasons_df['name'].str.extract("(\d+)-?(\d+)?")
seasons_df['start-year'] = yrs[0]
seasons_df['end-year'] = yrs[1]

seasons_df['Regular'] = ~seasons_df['name'].str.contains("KMT")
seasons_df.replace(pd.NA, None, inplace=True)

### Saving to SQlite

In [225]:
with sq.connect(DB_PATH) as conn:
    teams_df.to_sql("teams", conn, if_exists='replace', index=False) 
    clean_game_df.to_sql("games", conn, if_exists='replace', index=False) 
    seasons_df.to_sql("seasons", conn, if_exists='replace', index=False) 